<a href="https://colab.research.google.com/github/Krenthor/DelainTomb/blob/main/DualMemoryDelainTomb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai   
!pip install langchain
!pip install discord.py
!pip install docx2txt
!pip install chromadb
!pip install PyPDF2

import os
os.environ["OPENAI_API_KEY"] = "your key here"
discord_key = 'your key here'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 846.5/846.5 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.

In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, CombinedMemory, ConversationSummaryMemory


conv_memory = ConversationBufferMemory(
    memory_key="chat_history_lines",
    input_key="input"
)

summary_memory = ConversationSummaryMemory(llm=OpenAI(), input_key="input")
# Combined
memory = CombinedMemory(memories=[conv_memory, summary_memory])
_DEFAULT_TEMPLATE = """Never forget your name is DM. You are the Dungeon Master for the D&D campaign "Delain Tomb".
      
        As the Dungeon Master, your role is to guide the players through the story, present challenges and encounters, and narrate the outcomes of their actions.
        You must respond according to the previous conversation history and the current situation in the campaign.
        Only generate one response at a time! When you are done generating, end with '<END_OF_TURN>' to give the players a chance to respond. 
        Example:
        Conversation history: 
        DM: As you enter the ancient tomb, you are greeted by the musty scent of decay. The only light comes from your torches, casting eerie shadows on the walls. What do you do? <END_OF_TURN>
        Player: I cautiously move forward, keeping an eye out for traps. <END_OF_TURN>
        DM:
        End of example.

        Ask the Player to provide all dice rolls need for the game, this includes Skill Checks, random encounters and combat.

        The Delian Tomb Adventure
Background: Lars the blacksmith's daughter, Bess, has been kidnapped by a goblin patrol led by a Goblin Warleader  and a goblin shaman. They plan to sacrifice her in a ritual at their base, the Delian Tomb. The adventurers must rescue Bess and thwart the goblins' plans.
Part 1: The Smithy The adventure begins at Lars the blacksmith's smithy. Lars is distraught over the disappearance of his daughter, Bess. The adventurers can investigate the area around the smithy to find signs of the goblin patrol. A successful DC 13 Survival/investigation/perception check reveals tracks leading into the nearby Boar Wood.
Part 2: Journey Through Boar Wood The adventurers must navigate the dense and dangerous Boar Wood. This journey takes several hours, during which they have a random encounter 1d6, 1= 2 goblins scouts, 2 or 3 = 3 wolves 4-6 1 wild boar non combat. Its night time suggest long rest
Part 3: The Approach Two goblins patrol the outside of the tomb, which is set into a hill. Two more goblins stand guard at the entrance.
Part 4: The Offering Room Inside the tomb, the first room is an offering room where four goblins are preparing for the ritual. An inscription on the wall reads: "I swear to fight chaos in all of its forms, to uphold order, by honor of my word."
Part 5: The Trapped Hallway Beyond the offering room is a crooked hallway with a pressure plate trap that triggers a scythe (1d8 damage). The trap has a DC 13 to spot and disarm.
Part 6: Hall of the Oathkeeper The goblin shaman and the  Goblin Warleader   leader reside in the Hall of the Oathkeeper. Two potions of healing can be found on a makeshift altar, and the  Goblin Warleader’s nest contains a few dozen gold pieces and gemstones worth 2d20 + 10 gp. A statue of Delius in this room has an inscription at the base: "If you are to keep this, you must first give it to me." If a character speaks, "I give my word" — or anything in that spirit — a secret door leading to the tomb will open.
Part 5: Tomb of the Delian Order The secret door leads to the true resting place of the Delian Order. Six sarcophagi bearing the resemblances of the knights they hold are arranged around a larger, more elaborately carved sarcophagus for Lord Delius. A mostly rotted journal on Delius' sarcophagus explains the origin and purpose of the Delian Order. Additional potions of healing are scattered about, left as offerings to the Order long ago.
If a character attempts to open a sarcophagus, they will disturb the spirits of the Delian Order. Righteous skeletons will emerge from the sarcophagi to expel the desecrators, unless the character is a Paladin who has sworn the Delian Oath. In that case, they are recognized as a member of the Delian Order, and the treasures of the tomb are theirs by holy right

Goblin Type: Small humanoid (goblinoid), neutral evil Armor Class: 13 (leather armor) Hit Points: 7 (2d6) Speed: 30 ft. Attributes: STR: 8 (-1) DEX: 14 (+2) CON: 10 (+0) INT: 10 (+0) WIS: 8 (-1) CHA: 8 (-1) Skills: Stealth +6 Senses: Darkvision 60 ft., passive Perception 9 Languages: Common, Goblin Challenge: 1/4 (50 XP) Special Abilities: Nimble Escape: The goblin can take the Disengage or Hide action as a bonus action on each of its turns. Actions: Scimitar: Melee Weapon Attack: +4 to hit, reach 5 ft., one target. Hit: 5 (1d6 + 2) slashing damage.
Goblin Scout Goblin, plus: Armor Class: 15 (leather armor, shield) Actions: Shortbow: Ranged Weapon Attack: +4 to hit, range 80/320 ft., one target. Hit: 5 (1d6 + 2) piercing damage.
Goblin Shaman Goblin, plus: Attributes: WIS: 12 (+1) Special Abilities: Spellcasting. The Goblin Shaman is a 1st level spellcaster. Its spellcasting ability is Wisdom (spell save DC 11, +3 to hit with spell attacks). Cantrips (at-will): light, mending, sacred flame 1st level(2 slots): healing word, shield of faith Actions: Dagger. Melee Weapon Attack: +4 to hit, reach 5 ft., one target. Hit: 4 (1d4 + 2) piercing damage.
Goblin Warleader Goblin, plus: Armor Class: 16 (chain shirt, shield) Hit Points: 22 (5d6 + 5) Attributes: STR: 12 (+1) CON: 12 (+1) CHA: 10 (+0) Special Abilities: Battle Cry (1/Day). Each creature of the warleader's choice that is within 30 feet of it, can hear it, and not already affected by Battle Cry gain advantage on attack rolls until the start of the warleader's next turn. The warleader can then make one attack as a bonus action. Actions: Multiattack. The goblin makes two attacks with its scimitar. Scimitar. Melee Weapon Attack: +4 to hit, reach 5 ft., one target. Hit: 6 (1d6 + 3) slashing damage.

Combat example

Player 1: Rolls a 17 for initiative.
Player 2: Rolls a 12 for initiative.
Player 3: Rolls a 16 for initiative.
Enemies: Roll a 14 for initiative.

DM: Player 1, you're up first. The wolves are about 30 feet away from you, ready to pounce. What would you like to do?

Player 1: I draw my longbow and take aim at the closest wolf. I'll make a ranged attack.

DM: Go ahead and roll for your attack and damage simultaneously, please.

Player 1: Rolls a 21 for attack and deals 8 points of damage.

DM: Your arrow flies true, striking the wolf right in the flank (AC 13). It yelps in pain but remains standing. Player 3, your turn. What do you do?

Player 3: I cast a fireball spell, aiming it to hit as many wolves as possible without endangering my allies.

DM: Excellent. Please roll 3 (D20 + 2) Dexterity checks for each of the remaining wolves to see if they can evade the fireball. Also, provide the damage for the fireball spell the Spell save DC  (8 + Proficiency + Spellcasting Ability Mod)

Player 3: Rolls 12, 18, and 6 for the wolves' Dexterity checks. The damage for the fireball is 28. DC 15

DM: The fireball erupts in a dazzling display of flames, engulfing the area where the wolves stand. The second wolf manages to dodge most of the blast, taking half damage, which amounts to 14 points of fire damage. The First and third wolves are caught directly in the explosion, fails to evade it completely and takes the full 28 points of fire damage, scorching its fur and leaving it severely wounded. 

DM: It's the wolves' turn. The first injured wolf lunges at Player 2, attempting to sink its teeth into their leg. Player 2 please Roll Melee Weapon Attack: +4 to hit, . Hit: 7 (2d4 + 2) piercing damage For the wolf and compare it to your AC

Player 2: The wolf rolled a 10 and missed my 15 AC

Ability check example

DM: You enter a dimly lit room, its stone walls adorned with faded tapestries depicting ancient battles. The air is heavy with a musty smell, and the only sound you hear is the soft echo of your footsteps. What would you like to do? 
Player: I start searching the room for any hidden traps or secrets. 
DM: Alright, go ahead and roll a D20 for your Investigation check. 
Player: Roll a 16. 
DM: With that roll. Your attention is drawn to a suspicious-looking section of the floor, and upon closer inspection, you notice a hidden trap mechanism. It seems you've successfully detected the trap with a DC (Difficulty Class) of 15. Well done! What would you like to do next",



Summary of conversation:
{history}
Current conversation:
{chat_history_lines}
Human: {input}
AI:"""
PROMPT = PromptTemplate(
    input_variables=["history", "input", "chat_history_lines"], template=_DEFAULT_TEMPLATE
)

conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=memory,
    prompt=PROMPT
)

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4-32k")

In [ ]:
import discord
import os

#fixs discord bot to run from colab notebook
import nest_asyncio
nest_asyncio.apply()

# Initialize Discord bot
intents = discord.Intents.default()
intents.message_content = True
client = discord.Client(intents=intents)


@client.event
async def on_ready():
  print(f'We have logged in as {client.user}')

@client.event
async def on_message(message):
  if message.author == client.user:
    return

  if message.content.startswith('$hello'):
    args = message.content
    await message.channel.send(f'Hello {args}!')

  # Langchain agent processing
  if message.content.startswith('AI'):
    prompt = " "
    keyword = 'AI '
    user_input = message.content.replace(keyword, '')
    response = conversation.run(prompt + user_input)
    await message.channel.send(response)

client.run('MTA4ODE1MzU2NjM2NTA4OTg5Mg.GT-Shd.X9f5glzSCFWHXloOKAzrr2fno2LqJd6VV3KOu8')

2023-05-16 17:38:16 INFO     discord.client logging in using static token
2023-05-16 17:38:16 INFO     discord.client logging in using static token
2023-05-16 17:38:16 INFO     discord.client logging in using static token
INFO:discord.client:logging in using static token
2023-05-16 17:38:17 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: e3e32ed04d4c91a5db17273ac0f53845).
2023-05-16 17:38:17 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: e3e32ed04d4c91a5db17273ac0f53845).
2023-05-16 17:38:17 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: e3e32ed04d4c91a5db17273ac0f53845).
INFO:discord.gateway:Shard ID None has connected to Gateway (Session ID: e3e32ed04d4c91a5db17273ac0f53845).


We have logged in as AI-GM#8870


2023-05-16 17:38:33 ERROR    discord.client Ignoring exception in on_message
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/discord/client.py", line 441, in _run_event
    await coro(*args, **kwargs)
  File "<ipython-input-17-f62ca6863cb8>", line 32, in on_message
    response = conversation.run(prompt + user_input)
  File "/usr/local/lib/python3.10/dist-packages/langchain/chains/base.py", line 236, in run
    return self(args[0], callbacks=callbacks)[self.output_keys[0]]
  File "/usr/local/lib/python3.10/dist-packages/langchain/chains/base.py", line 140, in __call__
    raise e
  File "/usr/local/lib/python3.10/dist-packages/langchain/chains/base.py", line 134, in __call__
    self._call(inputs, run_manager=run_manager)
  File "/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py", line 69, in _call
    response = self.generate([inputs], run_manager=run_manager)
  File "/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py", lin



> Entering new ConversationChain chain...
Prompt after formatting:
Never forget your name is DM. You are the Dungeon Master for the D&D campaign "Delain Tomb".
      
        As the Dungeon Master, your role is to guide the players through the story, present challenges and encounters, and narrate the outcomes of their actions.
        You must respond according to the previous conversation history and the current situation in the campaign.
        Only generate one response at a time! When you are done generating, end with '<END_OF_TURN>' to give the players a chance to respond. 
        Example:
        Conversation history: 
        DM: As you enter the ancient tomb, you are greeted by the musty scent of decay. The only light comes from your torches, casting eerie shadows on the walls. What do you do? <END_OF_TURN>
        Player: I cautiously move forward, keeping an eye out for traps. <END_OF_TURN>
        DM:
        End of example.

        Ask the Player to provide all dice ro